In [ ]:
%reload_ext autoreload
%autoreload 2

current_jupyter_path: str = %pwd # type: ignore
while len(current_jupyter_path) > 1 and not current_jupyter_path.endswith("finance-tracker-agent"):
    %cd ..
    current_jupyter_path = %pwd # type: ignore

In [ ]:
from langchain.agents import (
    create_tool_calling_agent,
    AgentExecutor,
    create_react_agent,
)
from langchain_openai import ChatOpenAI
from src.tools import weather_tool, get_weather
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub


llm = ChatOpenAI(model="gpt-3.5-turbo")

## create_tool_calling_agent

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a weather assistant"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [ ]:
agent = create_tool_calling_agent(llm=llm, tools=[weather_tool], prompt=prompt)

executor = AgentExecutor(agent=agent, tools=[weather_tool], verbose=True)

result = executor.invoke({"input": "What is the weather in London?"})

print(result)

## create_react_agent

In [ ]:
prompt_template = hub.pull("hwchase17/react")  # ReAct-style reasoning prompt
tools = [weather_tool]


agent = create_react_agent(llm, tools, prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tools)


result = executor.invoke({"input": "What is the weather in Portugal?"})

**langgraph**

In [ ]:
from langgraph.prebuilt import create_react_agent


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a weather assistant. Use the provided tools to answer queries about the weather.",
        ),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [weather_tool]

agent = create_react_agent(model=llm, tools=tools, prompt=prompt)


result = agent.invoke(
    {"input": "What is the weather in Portugal?", "agent_scratchpad": ""}
)
result

## create_structured_chat_agent

In [ ]:
from langchain.agents import AgentExecutor, create_structured_chat_agent

prompt = hub.pull("hwchase17/structured-chat-agent")


agent = create_structured_chat_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke({"input": "hi"})

# Using with chat history
executor.invoke({"input": "What is the weather in Portugal?"})

## create_openai_functions_agent

In [ ]:
from langchain.agents import AgentExecutor, create_openai_functions_agent


prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [weather_tool]

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

result = agent_executor.invoke({"input": "What is the weather in Portugal?"})
result